In [1]:
import pandas as pd

import dataSrc
import utilities

# Retrieve symbols and financials

sp500_tickers = dataSrc.sp500_symbols()
dow_tickers = dataSrc.dow_symbols()

# sp500_financials = dataSrc.financials(sp500_tickers)

df = dataSrc.financials(dow_tickers)

# df = pd.read_csv('./R/data/dow_financials.csv')
# df = df.drop(['Unnamed: 0'], axis=1)
# df = df.dropna()

conditions = ((df["Revenue_Growth"] > 0)
              & (df["GPMargin"] > 0)
              & (df["EPS_Growth"] > 0)
              & (df["ROE"] > 0)
              & (df["DPS_Growth"] > 0)
              & (df["DivYield"] > 0))
df = df[conditions]

# Compute historical momentum
# Average momentum of prev 12M, 24M and 36M

m12_momentums = []
m24_momentums = []
m36_momentums = []
count = 0
for symbol in df["symbol"]:
    count += 1
    m12_momentums.append(utilities.calculate_hist_momentum(symbol, 252))
    m24_momentums.append(utilities.calculate_hist_momentum(symbol, 504))
    m36_momentums.append(utilities.calculate_hist_momentum(symbol, 756))

df["m12_momentum"] = m12_momentums
df["m24_momentum"] = m24_momentums
df["m36_momentum"] = m36_momentums


# watchlist = positive_mom[["symbol", "name"]].copy()

# # Update current prices, 52W High and Discount %

# currentPrices = []
# highs = []
# discounts = []
# count = 0

# for symbol in watchlist["symbol"]:
#     count += 1
#     print(f"{symbol}: {count}/{len(watchlist['symbol'])}")
#     currentPrice = utilities.get_current_price(symbol)
#     high = utilities.calculate_prev_max_high(symbol, 252)
#     discount_pct = (currentPrice - high) / high

#     currentPrices.append(currentPrice)
#     highs.append(high)
#     discounts.append(discount_pct)

# watchlist["CurrentPrice"] = currentPrices
# watchlist["52W_High"] = highs
# watchlist["Discount%"] = discounts

# watchlist = watchlist.sort_values(by="Discount%")

# watchlist

CRM: 1/30
WBA: 2/30
V: 3/30
NKE: 4/30
UNH: 5/30
TRV: 6/30
VZ: 7/30
INTC: 8/30
WMT: 9/30
JNJ: 10/30
DIS: 11/30
MCD: 12/30
JPM: 13/30
CAT: 14/30
BA: 15/30
AMGN: 16/30
DOW: 17/30
AAPL: 18/30
GS: 19/30
CSCO: 20/30
MSFT: 21/30
HD: 22/30
PG: 23/30
MRK: 24/30
IBM: 25/30
HON: 26/30
KO: 27/30
CVX: 28/30
AXP: 29/30
MMM: 30/30
   symbol                            name exchange                  sector  \
1     WBA  Walgreens Boots Alliance, Inc.   NASDAQ              Healthcare   
3     NKE                      NIKE, Inc.     NYSE       Consumer Cyclical   
5     TRV   The Travelers Companies, Inc.     NYSE      Financial Services   
6      VZ     Verizon Communications Inc.     NYSE  Communication Services   
7    INTC               Intel Corporation   NASDAQ              Technology   
9     JNJ               Johnson & Johnson     NYSE              Healthcare   
19   CSCO             Cisco Systems, Inc.   NASDAQ              Technology   
20   MSFT           Microsoft Corporation   NASDAQ        

In [4]:
positive_mom_stocks = df[(df['m12_momentum'] > 0) & (df['m24_momentum'] > 0)
                         & (df['m36_momentum'] > 0)]
positive_mom_stocks

,symbol,name,exchange,sector,industry,marketCap(B),Revenue_Growth,ROE,GPMargin,EPS_Growth,DivYield,DPS,DPS_Growth,m12_momentum,m24_momentum,m36_momentum
3,NKE,"NIKE, Inc.",NYSE,Consumer Cyclical,Footwear & Accessories,265.295905,0.595383,0.514070,0.448202,0.043478,0.006564,1.100,0.716614,0.492386,0.962755,1.029305
5,TRV,"The Travelers Companies, Inc.",NYSE,Financial Services,Insurance—Property & Casualty,40.440320,0.481540,0.132920,0.249376,0.280277,0.021161,3.430,0.317382,0.398654,0.103103,0.225355
7,INTC,Intel Corporation,NASDAQ,Technology,Semiconductors,218.631717,0.743302,0.231499,0.555936,0.506024,0.025478,1.373,0.374296,0.068610,0.149776,0.109533
9,JNJ,Johnson & Johnson,NYSE,Healthcare,Drug Manufacturers—General,455.234748,0.338716,0.270051,0.669963,0.012766,0.023940,4.140,0.348304,0.125553,0.348487,0.287448
19,CSCO,"Cisco Systems, Inc.",NASDAQ,Technology,Communication Equipment,248.722096,0.247370,0.266846,0.640210,0.048827,0.024737,1.460,0.433121,0.398578,0.248572,0.257082
20,MSFT,Microsoft Corporation,NASDAQ,Technology,Software—Infrastructure,2252.363006,1.328849,0.462317,0.689258,0.064902,0.007474,2.240,0.553834,0.309336,1.169997,1.718302
21,HD,"The Home Depot, Inc.",NYSE,Consumer Cyclical,Home Improvement Retail,341.282324,0.791112,7.047047,0.337181,0.159775,0.019482,6.300,1.358793,0.129554,0.422576,0.601129


In [2]:
#!jupyter nbconvert --to script --no-prompt analysis.ipynb